## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,carnarvon,AU,71.67,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
1,6,atuona,PF,79.65,light rain,-9.8000,-139.0333,Villa Enata
2,11,luanda,AO,75.20,broken clouds,-8.8368,13.2343,LANDU MARINEZA LUFUANDU TONI
3,21,boundiali,CI,73.24,broken clouds,9.5217,-6.4869,Hotel Paysan Boundiali
4,24,namie,JP,83.55,broken clouds,37.4833,141.0000,いこいの村なみえ


In [4]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-24.8667,113.6333
1,-9.8000,-139.0333
2,-8.8368,13.2343
3,9.5217,-6.4869
4,37.4833,141.0000
...,...,...
258,-1.7625,-48.8043
259,-15.7667,128.7333
260,45.0205,-93.2180
261,-17.7125,-39.2481


In [5]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
# 11b. Display the figure
fig = gmaps.figure(center=(30.1,31.1),zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, max_temp,dissipating=False,max_intensity=300,point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "sao jose da coroa grande"]

vacation_end = vacation_df.loc[vacation_df["City"] == "sao jose da coroa grande"]

vacation_stop1 = vacation_df.loc[vacation_df["City"] == "bom jardim"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "pitimbu"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "olinda"] 
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
69,220,sao jose da coroa grande,BR,73.98,clear sky,-8.8978,-35.1478,Crôa Mares Hotel


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.to_numpy()

#5,6 are the positions of Lat and Lng in the array of start
start_coords = (start[0][5],start[0][6]) 

end = vacation_end.to_numpy()
end_coords = (end[0][5],end[0][6])

stop1 = vacation_stop1.to_numpy()
stop1_coords = (stop1[0][5],stop1[0][6])

stop2 = vacation_stop2.to_numpy()
stop2_coords = (stop2[0][5],stop2[0][6])

stop3 = vacation_stop3.to_numpy()
stop3_coords = (stop3[0][5],stop3[0][6])

print(start_coords, end_coords,stop1_coords,stop2_coords,stop3_coords)

(-8.8978, -35.1478) (-8.8978, -35.1478) (-7.7958, -35.5872) (-7.4706, -34.8086) (-8.0089, -34.8553)


In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

import gmaps.datasets


fig = gmaps.figure()
start2end = gmaps.directions_layer(start_coords,end_coords,waypoints=[stop1_coords,stop2_coords,stop3_coords], travel_mode='DRIVING')
fig.add_layer(start2end)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,220,sao jose da coroa grande,BR,73.98,clear sky,-8.8978,-35.1478,Crôa Mares Hotel
1,269,bom jardim,BR,71.94,broken clouds,-7.7958,-35.5872,Pousada do Posto Jose Breno
2,615,pitimbu,BR,72.19,scattered clouds,-7.4706,-34.8086,Reserva do Abiaí
3,652,olinda,BR,73.13,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda


In [10]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map.
# Add a heat map for the vacation spots

max_temp = itinerary_df["Max Temp"]

fig = gmaps.figure(center=(-7,-35),zoom_level=7)
#marker for each city - where markers are lat,lng
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))